In [2]:
import os
os.chdir(r'C:\Users\Hp\Desktop\ml labs\timeseires')

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [4]:
class PC:
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        lstm=Bidirectional(LSTM(48,return_sequences=True))(inputs)
        lstm=LSTM(48)(lstm)
        lstm=Activation('tanh')(lstm)
        lstm=Dense(24)(lstm)
        # create the model
        model = Model(inputs, lstm, name="PC")
        
        # return the constructed network architecture
        return model

In [5]:
model1 =  PC.build(time_steps=24, num_features=21, reg=0.005)

In [6]:
model1.summary()

Model: "PC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 24, 96)           26880     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 48)                27840     
                                                                 
 activation (Activation)     (None, 48)                0         
                                                                 
 dense (Dense)               (None, 24)                1176      
                                                                 
Total params: 55,896
Trainable params: 55,896
Non-trainable params: 0
____________________________________________________________

In [7]:
checkpoints = r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\\E3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.json"])

In [8]:
model = None
start_epoch = 0
time_steps=24
num_features=21

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [10]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [20]:
import os
path_dataset =r'C:\Users\Hp\Desktop\ml labs\AEP hourly'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [21]:
time_steps=24
num_features=21

In [22]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=24)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=24)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=24)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.8065054416656494 sec


In [24]:
epochs = 1
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

2652/2652 [==============================] - ETA: 0s - loss: 0.0443 - mae: 0.0443 - mape: 2553.3105
Epoch 1: val_loss improved from 0.05004 to 0.04632, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0001-loss0.05.h5


PermissionDeniedError: Failed to create a directory: C:\Users/Administrator; Permission denied

In [22]:

model = load_model(r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0001-loss0.04.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

378/378 [==============================] - 10s 18ms/step
Mean Absolute Error (MAE): 623.13
Median Absolute Error (MedAE): 485.6
Mean Squared Error (MSE): 666817.93
Root Mean Squared Error (RMSE): 816.59
Mean Absolute Percentage Error (MAPE): 4.21 %
Median Absolute Percentage Error (MDAPE): 3.37 %


y_test_unscaled.shape=  (12082, 24)
y_pred.shape=  (12082, 24)


In [19]:
checkpoints = r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0001-loss0.04.h5'
start_epoch= 17

In [20]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E3-cp-0001-loss0.04.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [21]:
epochs = 2
verbose = 1 #0
batch_size = 128
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/2
663/663 [==============================] - ETA: 0s - loss: 0.0354 - mae: 0.0354 - mape: 2180.3840
Epoch 1: val_loss improved from inf to 0.03807, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E2-cp-0001-loss0.04.h5
663/663 [==============================] - 68s 89ms/step - loss: 0.0354 - mae: 0.0354 - mape: 2180.3840 - val_loss: 0.0381 - val_mae: 0.0381 - val_mape: 16.3709
Epoch 2/2
663/663 [==============================] - ETA: 0s - loss: 0.0351 - mae: 0.0351 - mape: 2173.3665
Epoch 2: val_loss improved from 0.03807 to 0.03756, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E2-cp-0002-loss0.04.h5
663/663 [==============================] - 55s 84ms/step - loss: 0.0351 - mae: 0.0351 - mape: 2173.3665 - val_loss: 0.0376 - val_mae: 0.0376 - val_mape: 16.1380


In [27]:

model = load_model(r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab2\E2-cp-0003-loss0.03.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

378/378 [==============================] - 7s 16ms/step
Mean Absolute Error (MAE): 503.62
Median Absolute Error (MedAE): 341.14
Mean Squared Error (MSE): 510885.31
Root Mean Squared Error (RMSE): 714.76
Mean Absolute Percentage Error (MAPE): 3.36 %
Median Absolute Percentage Error (MDAPE): 2.39 %


y_test_unscaled.shape=  (12082, 24)
y_pred.shape=  (12082, 24)
